In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [5]:
input_size=784
hidden_size=600
num_classes=10
num_epochs=5
batch_size=100
learning_rate = 0.001

In [8]:
train_dataset = torchvision.datasets.MNIST(root="./MNIST",
                                           train=True,
                                           download=True,
                                           transform=transforms.ToTensor())

In [9]:
test_dataset = torchvision.datasets.MNIST(root="./MNIST",
                                           train=False,
                                           download=True,
                                           transform=transforms.ToTensor())

In [10]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,shuffle=True,
                                          batch_size=batch_size)

In [11]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,shuffle=True,
                                          batch_size=batch_size)

In [23]:
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(NeuralNet,self).__init__()
        self.fc1=nn.Linear(input_size,hidden_size)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(hidden_size,num_classes)
    
    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [24]:
model = NeuralNet(input_size,hidden_size,num_classes).to(device)

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)


In [29]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs,labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

        

Epoch [1/5], Step [100/600], Loss: 0.0069
Epoch [1/5], Step [200/600], Loss: 0.0025
Epoch [1/5], Step [300/600], Loss: 0.0068
Epoch [1/5], Step [400/600], Loss: 0.0027
Epoch [1/5], Step [500/600], Loss: 0.0089
Epoch [1/5], Step [600/600], Loss: 0.0074
Epoch [2/5], Step [100/600], Loss: 0.0041
Epoch [2/5], Step [200/600], Loss: 0.0023
Epoch [2/5], Step [300/600], Loss: 0.0007
Epoch [2/5], Step [400/600], Loss: 0.0008
Epoch [2/5], Step [500/600], Loss: 0.0256
Epoch [2/5], Step [600/600], Loss: 0.0027
Epoch [3/5], Step [100/600], Loss: 0.0017
Epoch [3/5], Step [200/600], Loss: 0.0047
Epoch [3/5], Step [300/600], Loss: 0.0048
Epoch [3/5], Step [400/600], Loss: 0.0001
Epoch [3/5], Step [500/600], Loss: 0.0007
Epoch [3/5], Step [600/600], Loss: 0.0021
Epoch [4/5], Step [100/600], Loss: 0.0031
Epoch [4/5], Step [200/600], Loss: 0.0010
Epoch [4/5], Step [300/600], Loss: 0.0088
Epoch [4/5], Step [400/600], Loss: 0.0038
Epoch [4/5], Step [500/600], Loss: 0.0016
Epoch [4/5], Step [600/600], Loss:

In [30]:
with torch.no_grad():
    correct = 0
    total =0
    for images,labels in test_loader:
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)
        
        outputs=  model(images)
        _,predicted = torch.max(outputs,1)
        total +=labels.size(0)
        correct += (predicted==labels).sum()
    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


Accuracy of the network on the 10000 test images: 98.11000061035156 %


In [31]:
# Save the model checkpoint
torch.save(model.state_dict(),"forward_neural.ckpt")